<a href="https://colab.research.google.com/github/neuralfalcon/piper-tts/blob/main/Piper_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[piper github repository](https://github.com/rhasspy/piper/tree/master)


In [9]:
from IPython.display import clear_output
%cd /content/
!wget https://raw.githubusercontent.com/neuralfalcon/piper-tts/main/best_quality.json
# https://github.com/neuralfalcon/piper-tts
!pip install piper-tts

import os
if os.path.exists("/content/audio"):
    print("audio folder already exists")
else:
    os.mkdir("/content/audio")
if os.path.exists("/content/model"):
    print("model folder already exists")
else:
    os.mkdir("/content/model")

import uuid
import re

def format_name(text):
    if text.endswith("."):
        text = text[:-1]

    text = text.lower()
    text = text.strip()

    # Replace non-alphabetic characters with a single underscore
    text = re.sub('[^a-z]+', '_', text)

    return text

# Example usage:
input_text = "This is / a, Sample! Text."
formatted_text = format_name(input_text)
print(formatted_text)

def tts_file_name(text):
    text = format_name(text)
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/audio/{truncated_text}_{random_string}.wav"
    return file_name



import subprocess
def piper_tts(text,model_name):
    if os.getcwd() != "/content/model":
      os.chdir("/content/model")
    output_file=tts_file_name(text)
    command = f'echo "{text}" | piper --model {model_name} --output_file {output_file}'
    result =subprocess.run(command, shell=True)
    if result.returncode == 0:
        print("TTS Generated Successfully !")
        return output_file
    else:
        print(f"Command failed with return code {result.returncode}")
        return None




import ipywidgets as widgets
from IPython.display import display
import json
# Assuming data is the content of your result.json
with open("/content/best_quality.json", 'r', encoding='utf-8') as f:
    read_data = json.load(f)
# Extracting language names, voice names, and model names
languages = list(read_data.keys())
voice_info = {lang: list(read_data[lang].keys()) for lang in languages}
model_info = {lang: {voice: read_data[lang][voice]["model_name"] for voice in voice_info[lang]} for lang in languages}




clear_output()
#@title Select Language and voice Model { display-mode: "form" }

import ipywidgets as widgets



# Function to update the available voices based on the selected language
def update_voices(change):
    selected_language = change.new
    if selected_language in voice_info:
        voice_names.options = voice_info[selected_language]
        voice_names.value = voice_info[selected_language][0]

# Create and display the language dropdown
sorted_language_list = sorted(languages, key=lambda x: x.split(' ')[0])
language_name = widgets.Dropdown(options=sorted_language_list, value='English (English, United States)')
display(language_name)

# Create and display the initial voice dropdown
initial_language = language_name.value
initial_voices = voice_info.get(initial_language, [])
# can i sort initial_voices alphabaticaly base on first chartecter
sorted_initial_voices = sorted(initial_voices)
voice_names = widgets.Dropdown(options=sorted_initial_voices, value=sorted_initial_voices[0])
display(voice_names)

# Attach the callback function to the language dropdown's observe method
language_name.observe(update_voices, names='value')


Dropdown(index=8, options=('Arabic (Arabic, Jordan)', 'Catalan (Catalan, Spain)', 'Chinese (Chinese, China)', …

Dropdown(options=('amy', 'arctic', 'danny', 'hfc_male', 'joe', 'kathleen', 'kusal', 'l2arctic', 'lessac', 'lib…

In [14]:
Enter_Text = 'piper tts is the best'  # @param {type: "string"}
model_name=model_info[language_name.value][voice_names.value]
save_tts_path=piper_tts(Enter_Text,model_name)
print(f"Language : {language_name.value}")
print(f"Voice Name : {voice_names.value}")
print(f"Voice Model Name : {model_name}")
print(f"TTS Save at : {save_tts_path}")
from IPython.display import Audio
Audio(save_tts_path,autoplay=True)


TTS Generated Successfully !
Language : English (English, United States)
Voice Name : kusal
Voice Model Name : en_US-kusal-medium
TTS Save at : /content/audio/piper_tts_is_the_best_446BD9A7.wav


In [12]:
#@title Download tts audio { display-mode: "form" }
from google.colab import files
if save_tts_path!=None:
  files.download(save_tts_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# !piper --help